### *Please note that this notebook should be run in a VertexAI managed notebook instance to support the BigQuery integration syntax.*

# ELT in BigQuery

We saw how to process Data in Apache Beam & Dataflow following ETL (Extract-Tranform-Load) principles.

Next we are taking a look at ETL (Extract Transform Load) using the BigQuery Data Warehouse.

ELT trades off slower speed of on demand insights for lower storage cost. Typically ELT makes sense when analyzing large data volumes.

## Create BigQuery Dataset

To kick things off we create a BigQuery dataset to accomodate our data.

In [1]:
from google.cloud import bigquery

try: 
    client = bigquery.Client() # Constructing BQ client object.

    dataset_id = "{}.data_journey_elt".format(client.project) # Defining dataset ID.
    dataset = bigquery.Dataset(dataset_id)

    dataset.location = "US" # Defining dataset location

    dataset = client.create_dataset(dataset, timeout=30)  # Creating dataset by API requets.
    print("Created dataset {}.{}".format(client.project, dataset.dataset_id))
except: 
    print('Creation failed. Dataset may already exist.')

Creation failed. Dataset may already exist.


Next, we access the public firebase dataset using the BigQuery integration.

We will explore App User behaviour to demonstrate the ELT concepts. The Extraction step is in this example already finished as the raw data is available for us to analyze.

#@bigquery
SELECT *
FROM `firebase-public-project.analytics_153293282.events_*`
LIMIT 3

## Transformation & Feature Engineering Using BigQuery Views

[BigQuery Views](https://cloud.google.com/bigquery/docs/views-intro) allow mapping transformations over a dataset without processing the actual result. 

### View #1

To keep track of users that churned we create boolean features for 'churn' & 'bounce'.

#@bigquery
CREATE OR REPLACE VIEW data_journey_elt.user_returninginfo AS
WITH firstlasttouch AS (
    SELECT
      user_pseudo_id,
      MIN(event_timestamp) AS user_first_engagement,
      MAX(event_timestamp) AS user_last_engagement
    FROM
      `firebase-public-project.analytics_153293282.events_*`
    WHERE event_name="user_engagement"
    GROUP BY
      user_pseudo_id

  )
  SELECT
    user_pseudo_id,
    user_first_engagement,
    user_last_engagement,
    EXTRACT(MONTH from TIMESTAMP_MICROS(user_first_engagement)) as month,
    EXTRACT(DAYOFYEAR from TIMESTAMP_MICROS(user_first_engagement)) as julianday,
    EXTRACT(DAYOFWEEK from TIMESTAMP_MICROS(user_first_engagement)) as dayofweek,

    (user_first_engagement + 86400000000) AS ts_24hr_after_first_engagement,

IF (user_last_engagement < (user_first_engagement + 86400000000),
    1,
    0 ) AS churned,

IF (user_last_engagement <= (user_first_engagement + 600000000),
    1,
    0 ) AS bounced,
  FROM
    firstlasttouch
  GROUP BY
    1,2,3

### View #2

To keep track of user demographics.

#@bigquery
CREATE OR REPLACE VIEW data_journey_elt.user_demographics AS
WITH first_values AS (
    SELECT
        user_pseudo_id,
        geo.country as country,
        device.operating_system as operating_system,
        device.language as language,
        ROW_NUMBER() OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp DESC) AS row_num
    FROM `firebase-public-project.analytics_153293282.events_*`
    WHERE event_name="user_engagement"
    )
SELECT * EXCEPT (row_num)
FROM first_values
WHERE row_num = 1

### View 3

In view #3 we aggregate certain user behaviour events.

#@bigquery
CREATE OR REPLACE VIEW data_journey_elt.user_aggregate_behaviour AS
WITH events_first24hr AS (
    SELECT
      e.*
    FROM
      `firebase-public-project.analytics_153293282.events_*` e
    JOIN
      data_journey_elt.user_returninginfo r
    ON
      e.user_pseudo_id = r.user_pseudo_id
    WHERE
      e.event_timestamp <= r.ts_24hr_after_first_engagement
    )
SELECT
  user_pseudo_id,
  SUM(IF(event_name = 'user_engagement', 1, 0)) AS cnt_user_engagement,
  SUM(IF(event_name = 'level_start_quickplay', 1, 0)) AS cnt_level_start_quickplay,
  SUM(IF(event_name = 'level_end_quickplay', 1, 0)) AS cnt_level_end_quickplay,
  SUM(IF(event_name = 'level_complete_quickplay', 1, 0)) AS cnt_level_complete_quickplay,
  SUM(IF(event_name = 'level_reset_quickplay', 1, 0)) AS cnt_level_reset_quickplay,
  SUM(IF(event_name = 'post_score', 1, 0)) AS cnt_post_score,
  SUM(IF(event_name = 'spend_virtual_currency', 1, 0)) AS cnt_spend_virtual_currency,
  SUM(IF(event_name = 'ad_reward', 1, 0)) AS cnt_ad_reward,
  SUM(IF(event_name = 'challenge_a_friend', 1, 0)) AS cnt_challenge_a_friend,
  SUM(IF(event_name = 'completed_5_levels', 1, 0)) AS cnt_completed_5_levels,
  SUM(IF(event_name = 'use_extra_steps', 1, 0)) AS cnt_use_extra_steps,
FROM
  events_first24hr
GROUP BY
  1
